# To Do

List of things to-do

- [ ] Checkout Gridap.jl/test/ODEsTests/DiffEqsWrappersTests [link](#s002_1)


---

### DiffEqsWrappersTests <a id='s002_1'> </a>

There is another package called `differentialEqs` or something like that which is useful for solving higher order time-deriv and RK4 time-stepping methods and so on. <br>
Gridap has a wrapper for this library. <br>
Checkout the tests for this wrapper and see if you can use this library.

---

---
---

# Note

### Note-1: Weak-form, the variables are functions

Consider the weak form of the NLSWE. <br>
In Gridap it will be written in the following form.

In [ ]:
res(t, (η, u), (ψ1, ψ2)) =
        ∫( ∂t(η)*ψ1 - (∇(ψ1)⋅(u*(h0+η))) )*dΩ + 
        ∫( ∂t(u)⋅ψ2 + (∇(u)'⋅u)⋅ψ2 + (∇(η)⋅ψ2)*g )*dΩ

Here, each of the variables such as &eta;, h0, u, represent a function which will interpolate the repective value onto a continous space, based on the FESpace. <br>
Hence, the weak form is written so as to compute the value of the residual at the Gauu quadrature point.

For Example, consider the following simple weak form term.
$$
\int_\Omega  u  d\Omega
$$

Here the procedure in the code will be following

- Go to the quadrature point `qp` in the cell
- Calculate the value $u(qp) = \sum_i Q_i u_i$ <br> where $u_i$ corresponds to the values of u at the vertices and $Q_i$ is the quadrature weight. 
- Therefore, the term in the weak form corresponds to a function `u` which will calculate the required value at the quadrature point.

Hence, if you have to evluate the following term
$$
\int_\Omega  u(h+\eta)  d\Omega
$$
then you have to think along the following way.

- Go to the quadrature point `qp` in the cell
- Calculate the value $u(qp) = \sum_i Q_i u_i (h_i + \eta_i)$ <br> where $u_i$, $h_i$, $\eta_i$ corresponds to the values of u, h, &eta; at the vertices and $Q_i$ is the quadrature weight. 
- Hence the equivalent Gridap weak form will be  <br> `res(t, (η, u), (ψ1, ψ2)) = ∫( u*(h0+η) )*dΩ `
- The above procedure requires that u, h0 and &eta; are FESpaceFieldFnc. Only then will it be able to construct the required interpolation function to evluate the value at the quadrature point.
- This form is hence very simple. You do not need to think along the lines of arrays containing FEM nodes or anything like that.

---
---

# Gridap Tutorials

The following tutorials were important

1. t015_interpolation_fe 
    - Needed for learning how to do interpolation of a FESpace at any random point. 
    - Also required for interpolating between two different FESpaces.
        - This will be useful in formulating random bathymetry problems. <br>
1. t001_poisson
    - Poisson eqn soln
1. t002_validation
    - Understanding convergence analysis calls
    - Basics of plotting
1. t017_poisson_transient
    - Soln of transient problems

# List of Gridap Methods, Types and Functions 

Listing down the various methods (M), types (T) and functions (f) from Julia as I encounter them.

|  SN | call | Desc | Remarks |
| --- | ---- | ---- | ------- |
| T-001 |  DiscreteModelFromFile("filename.json") | Import FE Mesh from file |   |
| T-002 | ReferenceFE(lagrangian,Float64,order) | Sets the type of the test / trial fnc |   |
| F-001 | TestFESpace | Results in the Test function | Args are (model, refFe; conformity=:H1, dirichlet_tags="sides") |
| F-002 | TrialFESpace | Results in the trial function |  |
| T-003 | Triangulation() <br> same as <br> Interior() | Create an object for formulating the numerical integration |  |
| T-004 | BoundaryTriangulation() <br> same as <br> Boundary() | Create an object for formulating the numerical integration at boundary faces |  |
| T-008 | Skeleton | For a space in dimension D, it returns the non-bnd entities of dimension d-1 | **NOTE** <br>Skeleton is more than a Triangulation! It contains a left and a right triangulation, hence useful for the jump (discontinuity) terms |
| T-005 | Measure | Object with domain integ for each element, based on the quadrature rule that you specify. <br> For the quadrature degree check this [reference](https://en.wikipedia.org/wiki/Gaussian_quadrature)  | arg(TriangulationObj, degree) |
| T-006 | AffineFEOperator | Sets up the AX=B for the linear FE problem. |  |
| F-003 | &nabla; | A Gridap function that associates the function f with gradient of f |  |
| T-007 | CartesianDiscreteModel | Available from Gridap. Generate a Cartesian grid |  |

T-008 <br> F-003

---
---

# List of Macros

|  SN | call | Desc | Remarks |
| --- | ---- | ---- | ------- |
| Ma-001 | @assert |   |   |
| Ma-002 | @check |   |   |
| Ma-003 | @eval  |   |   |
| Ma-003 | @inbounds  |   |   |
| Ma-004 | @test |   |   |
| Ma-005 | @notimplemented |   |   |


---
---

# Math and Algebra Operations

## Backslash operator A \ b

Refere to this [link](https://stanford.edu/class/engr108/lectures/julia_inverses_slides.pdf) regarding the backslash operator and its functions when `A` is a square, tall or wide matrix in `Ax = b`

**Note**: `A` does not have to be square!

|     |
| --- |
| <img width="100%" src="./img/s002_p01_backslash01.png" /> |
| <img width="100%" src="./img/s002_p01_backslash02.png" /> |
|  |


---


## Function composition (Adding two functions)

Check this important feature of Julia. <br>
https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping <br>
It is written as `\circ`. <br>

This was crucial for some operations. <br>
For example, while running `BeamMultJoints_freq.jl`, I was getting the final soln with the following statement, where all quantities `(ϕₕ,κₕ,ηₕ)` are CellFieldFunctions with complex values `(a+ib)`. <br>
But when I tried to get the angle of the complex number using `angle(ϕₕ)`, it gave me the following error.
```
LoadError: MethodError: no method matching angle(::Gridap.FESpaces.SingleFieldFEFunction{Gridap.CellData.GenericCellField{Gridap.CellData.ReferenceDomain}})
Closest candidates are:
  angle(::Real) at number.jl:214
  angle(::Complex) at complex.jl:627
```

This is because `angle()` was not defined for CellFieldFunction. <br>
So instead you can use `angle∘(ϕₕ)`.
The composition operator then forces angle and the lazy map of CellFieldFunction to evaluate the quantities at every node and hence works.

---

---
---

# Geometry elements

Refer to `t003_elasticity.ipynb` regarding the commands in this section.

Use the following command `get_face_labeling` to get list of elements with different number of face.

In [3]:
using Gridap
using Gridap.Geometry

model = DiscreteModelFromFile("../models//solid.json")

labels = get_face_labeling(model)

FaceLabeling:
 0-faces: 10257
 1-faces: 57596
 2-faces: 87482
 3-faces: 40144
 tags: 6
 entities: 611

Here the 0 faces indicate (I think) the number of nodes, 1 face is edges, 2 face is sides and 3-face is the elements.

If you want to list down all entities with 3-faces then do the following.

In [4]:
tags = get_face_tag(labels, 3)

40144-element Vector{Int8}:
 5
 5
 5
 5
 5
 5
 5
 5
 5
 5
 ⋮
 6
 6
 6
 6
 6
 6
 6
 6
 6

Previous lines generate a vector, namely `tags`, whose length is the number of cells in the model and for each cell contains an integer that identifies the material of the cell. <br>
The tutoruial was meant to use two different materials (alu and steel) in different cells, hence each cell is marked with two different integer value to identify them as alu or steel.

---
---